In [ ]:
import pandas as pd
from pandas import *
import numpy as np
from numpy import *
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from scipy.optimize import curve_fit
from sklearn.linear_model import *
from sklearn.feature_selection import RFE
from sklearn.model_selection import *
from sklearn.tree import *
import forestci as fci

#loading features from the csv-files
features = pd.read_csv('Data_set_Hsp90_features_protein-ligand-complex.csv', ';', usecols=['6.6','7.6','8.6','16.6','53.6','6.7','7.7','8.7','16.7','6.8','7.8','8.8','16.8','53.8','6.9','7.9','8.9','9.9','15.9','16.9','6.15','6.16','7.16','8.16','16.16','6.17','7.17','8.17','16.17','6.35','7.35','8.35','15.35','16.35','logkoff'])
features.head(72)
features = pd.get_dummies(features, prefix_sep='_', dtype=float)#for the smiles code ->(0/1)
features.iloc[:,:].head(72)
labels = np.array(features['logkoff'])#labels value we want to predict
features= features.drop('logkoff', axis = 1)#remove the label, axis=1 is the column
feature_list = list(features.columns) #saving for later
features = np.array(features)

# Using Skicit-learn to split data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 7)


#Building the RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 250,max_depth = 30,max_features = 'log2', random_state = 7)
rf.fit(train_features, train_labels)
predictions = rf.predict(test_features)
errors = abs(predictions - test_labels)

#Calculate different metrics for comparing the ML model
print('Mean Absolute Error in koff:',round(np.mean(errors), 5), 'per second.')
predicted = rf.predict(train_features)
expected = train_labels

numerator = ((test_labels - predictions) ** 2).sum()
denominator = ((test_labels - np.average(test_labels)) ** 2).sum()
r2_score1 = 1 - (numerator / denominator)
print('R2-score test-set:',round(r2_score1,3))

numerator = ((expected - predicted) ** 2).sum()
denominator = ((expected - np.average(expected)) ** 2).sum()
r2_score2 = 1 - (numerator / denominator)
print('R2-score train-set:',round(r2_score2,3))


import math
 
MSE = np.square(np.subtract(test_labels,predictions)).mean() 
 
RMSE = math.sqrt(MSE)
print("RMSE test-set:", round(RMSE,3))

MSEt = np.square(np.subtract(expected,predicted)).mean() 
 
RMSEt = math.sqrt(MSEt)
print("RMSE train-set:", round(RMSEt,3))


from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(test_labels, predictions)
print('MAE test-set:',round(MAE,3))

from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(expected, predicted)
print('MAE train-set:',round(MAE,3))

#plot the predicted and expected datapoints with errorbars
plt.style.use('ggplot')
predictions_data = pd.DataFrame(data = {'koff': test_labels, 'prediction': predictions})
x = test_labels
y = predictions_data['prediction']
mpg_V_IJ_unbiased = fci.random_forest_error(rf, train_features, test_features)
plt.errorbar(x, y, yerr=np.sqrt(mpg_V_IJ_unbiased), fmt = '.r')
plt.plot([-4.5, 0.0], [-4.5, 0], linestyle = '--', color = 'grey')
plt.plot(x,y,'s',markersize = 7.5, label='test set, $R^2$=0.860', color = 'red')
plt.plot(expected, predicted, '^',markersize = 7.5, label='training set, $R^2$=0.934', color ='green')
plt.xlim(0,-4.5)
plt.ylim(0,-4.5)
plt.xlabel('experimental log koff (log s-1)')
plt.ylabel('predicted log koff (log s-1)')
plt.legend()
plt.title('Experimental and Predicted Values')
plt.show()

#caluclating the feature importance with permutation
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
r = permutation_importance(rf, test_features, test_labels, n_repeats=30, random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
         print(f"{feature_list[i]:24}"
               f"{r.importances_mean[i]:.3f}"
               f" +/- {r.importances_std[i]:.3f}")

#sort the features with the standard deviation after the importance    
a = (list(feature_list))
b = list(r.importances_mean)
c = list(r.importances_std)
my_dict={}
for i in range(len(a)):
    my_dict[a[i]]= (b[i],c[i])

sorted_dictonary=sorted(my_dict.items(), key=lambda x: x[1])
sorted_keys=[k for k, v in sorted_dictonary]
sorted_errors=[v[1] for k, v in sorted_dictonary]
sorted_importance=[v[0] for k, v in sorted_dictonary]

#plot the calculated importance of the features
plt.style.use('ggplot')
x_values = list(range(len(a)))
plt.bar(sorted_keys, sorted_importance, orientation = 'vertical', color = 'darkblue', facecolor=None, yerr=sorted_errors)
plt.xticks(x_values, rotation='vertical', fontsize=11)
plt.yticks(fontsize=11)
plt.ylabel('Importance'); plt.xlabel('Feature'); plt.title('Feature importances measured with permutation')
plt.show()